## Core 4: Pollard Rho Method

In [1]:
import numpy as np
import numba
import time

In [2]:
def gcd(a,b):
    """Returns the greatest common divisor of integers a and b using Euclid's algorithm.
    The order of a and b does not matter and nor do the signs."""
    if not(a%1 ==0 and b%1==0):
        return "Need to use integers for gcd."
    if b==0:
        return abs(a)                           #Use abs to ensure this is positive
    else:
        return gcd(b,a%b)

def g(a):
    return (a**2 + 1)

In [3]:

def pollard_rho99(n):
    x = 2
    y = 2
    d = 1

    while d == 1:
        x = g(x)%n
        y = g(g(y))%n
        d = gcd(np.abs(x - y), n)
    
    if d == n: 
        return failure
    else:
        return d
    
pollard_rho99(396421778273)





609071

## Faster Version

In [4]:
@numba.jit('int64(int64)', nopython = True, cache = True)
def g_fast(a):
    return (a**2 + 1)


@numba.jit('int64(int64,int64)', nopython=True, cache = True)
def gcd_fast(a, b):
    """Returns the greatest common divisor of integers a and b using Euclid's algorithm.
    The order of a and b does not matter and nor do the signs."""
    if not(a % 1 == 0 and b % 1 == 0):
        return -1
    if b == 0:
        return abs(a)  # Use abs to ensure this is positive
    else:
        return gcd_fast(b, a % b)

@numba.jit('int64(int64)', nopython=True, cache = True)
def pollard_rho99_fast(n):
    x = 2
    y = 2
    d = 1
        
    while d == 1:
        x = g_fast(x) % n
        y = g_fast(g_fast(y)) % n
        d = gcd_fast(np.abs(x - y), n)

    if d == n:
        return -1
    else:
        return d
    
    
@numba.vectorize('int64(int64)', nopython = True, target = 'parallel', cache = True)
def pollard_rho99_fast_ufunc(n):
    return pollard_rho99_fast(n)
    

In [5]:
pollard_rho99_fast(396421778273)

609071

## Compare

In [6]:
a = np.random.randint(low = 10, high = 40000000, size =10, dtype = 'int64')
%timeit [pollard_rho99(i) for i in a]
%timeit [pollard_rho99_fast(i) for i in a]
%timeit pollard_rho99_fast_ufunc(a)

135 µs ± 2.09 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
4.36 µs ± 57.7 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
2.81 µs ± 2.89 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [7]:
pollard_rho99_fast(396421778273)

%timeit pollard_rho99(396421778273)
%timeit pollard_rho99_fast(396421778273)
%timeit pollard_rho99_fast_ufunc(396421778273)

16.4 ms ± 47.3 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
57.9 ms ± 260 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
59.1 ms ± 851 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


So it runs faster when doing big lot of calculations